In [1]:
from bs4 import BeautifulSoup
import re
import xmlToSql

In [2]:
with open('frharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')

In [3]:
pages = soup.find_all('page')

In [6]:
with open('personnages.txt','x') as f:
    for page in pages:
        if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
            f.write(xmlToSql.getAttribute("Nom",page.text) +" "+ xmlToSql.getAttribute("Yeux",page.text)+"\n")

AttributeError: module 'xmlToSql' has no attribute 'getAttribute'

In [42]:
def getAttribute(attribute, target):
    try:
        result = re.search(f'\|{attribute}=([\w\ 0-9,\']*)\W.*?\s\|', target).group(1)
        if result == "":
            return "Inconnu"
    except:
        return "Inconnu"
    return result

def getNaissance(target):
    try:
        internal = re.search('\|Naissance\ *?=\ *?([\[\[.*\]\]|\{\{.*?\}\}]).*?\s\|',target).group(1)
    except:
        if re.search(f'\|Naissance=()\s\|',target) is not None:
            return "Inconnu"
        else:
            return "ERROR"
    results = re.findall('\[\[(.*?)\]\]|\{\{(.*?)\}\}',internal)
    print(results)
    result = ""
    for res in results:
        result += res[0]+res[1] + " "
    return result

# with open('personnagesnaissance.txt','x') as f:
#     for page in pages:
#         if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
#             f.write(getAttribute("Nom",page.text) +" "+ getAttribute("Naissance",page.text)+"\n")
for page in pages[0:100]:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        print(getAttribute("Nom",page.text) +" "+ getNaissance(page.text))

[]
Harry Potter 
[]
Drago Malefoy 
[]
Ronald Weasley 
[]
Albus Dumbledore 
[]
Tom Elvis Jedusor 
Alastor Maugrey ERROR
[]
Minerva McGonagall 


## Creating the database

The database will be of the following structure:



1
2
0.3755656108597285


In [27]:
string = "|Yeux=Marron&lt;ref&gt;{{Référence|2x13}}&lt;/ref&gt;|Cheveux=Bruns&lt;ref&gt;{{Référence|1x06}}&lt;/ref&gt;|Naissance=[[19 septembre]] [[1979]]&lt;ref name=\"naissance\"&gt;{{Référence|Pottermore}}&lt;/ref&gt;\n|Mort=|Famille=*[[Père d'Hermione Granger]] {{Comment|père}}*[[M"
internal = re.search('\|Naissance\ *?=\ *?(\[\[.*\]\]).*?\s\|',string).group(1)
results = re.findall('\[\[(.*?)\]\]',internal)
result = ""
for res in results:
    result += res + " "
print(result)

19 septembre 1979 
